In [1]:
from netCDF4 import Dataset
from scipy.interpolate import interp1d
import sys
import numpy as np


进行MPAS垂直插值的测试note脚本

In [2]:
filename = 'history.1998-04-01_00.00.00.nc'
outfile = 'isobaric.nc'

#
# Set list of isobaric levels (in hPa)
#
levs_hPa = [ 1000.0, 975.0, 950.0, 925.0, 900.0,
              850.0, 800.0, 750.0, 700.0, 650.0,
              600.0, 550.0, 500.0, 450.0, 400.0,
              350.0, 300.0, 250.0, 200.0, 150.0,
              100.0,  70.0,  50.0,  30.0,  20.0,
               10.0 ]

field_names = [ 'uReconstructZonal', 'uReconstructMeridional' ]

#
# Set the fill value to be used for points that are below ground
# or above the model top
#
fill_val = -1.0e30

In [3]:
#
# Read 3-d pressure, zonal wind, and meridional wind fields
# on model zeta levels
#
f = Dataset(filename)

nCells = f.dimensions['nCells'].size
nVertLevels = f.dimensions['nVertLevels'].size

pressure = f.variables['pressure'][:]

xtime = f.variables['xtime'][:]

fields = []
for field in field_names:
    fields.append(f.variables[field][:])

f.close()

In [5]:
#
# Convert pressure from Pa to hPa
#
pressure = pressure * 0.01


#
# Compute logarithm of isobaric level values and 3-d pressure field
#
pressure = np.log(pressure)
levs = np.log(levs_hPa)
nlevs_hPa = levs.shape[0]
# levs_hPa
#
# Allocate list of output fields
#
isobaric_fields = []
isobaric_fieldnames = []
for field in field_names:
    # for lev in levs_hPa:
    isobaric_fields.append( np.zeros([1, nCells, nlevs_hPa]) )
    isobaric_fieldnames.append(field)



In [7]:
# 插值测试
y = interp1d(pressure[0,0,:], fields[0][0,0,:], bounds_error=False, fill_value=fill_val)
levs
# levs_hPa[6]
scipy_res = y(levs[:])
numpy_res = np.interp(levs, pressure[0,0,::-1], fields[0][0,0,::-1] ,left=fill_val, right=fill_val)
numpy_res

array([-1.00000000e+30, -1.00000000e+30, -1.00000000e+30, -1.69644935e+00,
       -2.33443657e+00, -1.29093947e+00,  1.62345028e+00,  4.01263529e+00,
        6.41309163e+00,  1.29645191e+01,  2.13300091e+01,  2.70363041e+01,
        3.03460612e+01,  2.99743115e+01,  2.90260636e+01,  2.90997202e+01,
        3.24507962e+01,  4.13106362e+01,  4.60601588e+01,  3.96995083e+01,
        2.96115501e+01,  2.51929696e+01,  2.61472343e+01,  2.35847630e+01,
        2.29389566e+01, -1.00000000e+30])

In [35]:
#
# Create netCDF output file
#


f = Dataset(outfile, 'w', clobber=False)

f.createDimension('Time', size=None)
f.createDimension('nCells', size=nCells)
f.createDimension('plevels', size=nlevs_hPa)
for field in isobaric_fieldnames:
    # print(field.shape)
    f.createVariable(field, 'f', dimensions=('Time','nCells', 'plevels'), fill_value=fill_val)


#

In [15]:
# np.interp(levs, pressure[0,0,::-1], field[0,0,::-1] \
            # ,left=fill_val, right=fill_val)
# numpy_res = np.interp(levs, pressure[0,0,::-1], fields[0][0,0,::-1] ,left=fill_val, right=fill_val)
# numpy_res
# pressure[0,0,::-1]
# field[0,0,::-1] 

In [25]:
# isobaric_fields[i][t,iCell,:] = np.interp(levs, pressure[t,iCell,::-1], field[t,iCell,::-1],\
#             left=fill_val, right=fill_val)
# isobaric_fields[0].shape

(1, 163842, 26)

In [16]:
for t in range(len(xtime)):

    timestring = xtime[t,0:19].tostring()
    print('Interpolating fields at time '+timestring.decode('utf-8'))

    #
    # Vertically interpolate
    #
    print("----- start interpolation -----")
    for iCell in range(nCells):

        i = 0
        for field in fields:
            
            isobaric_fields[i][t,iCell,:] = np.interp(levs, pressure[t,iCell,::-1], field[t,iCell,::-1],\
            left=fill_val, right=fill_val)
            i = i + 1



    print("----- finish interpolation -----")




<ipython-input-16-c8013cd75365>:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  timestring = xtime[t,0:19].tostring()
Interpolating fields at time 1998-04-01_00:00:00
----- start interpolation -----
----- finish interpolation -----


In [34]:
isobaric_fieldnames[1]
# isobaric_fields[i][t,:,:]

'uReconstructMeridional'

In [40]:
#
# Save interpolated fields
#
for i in range(len(isobaric_fieldnames)):
    # print("---write---")
    # print(i)
    f.variables[isobaric_fieldnames[i]][t,:,:] = isobaric_fields[i][t,:,:]
f.close()

In [44]:
f = Dataset(outfile, 'r', clobber=False)
f.variables[isobaric_fieldnames[0]][:]

masked_array(
  data=[[[--, --, --, ..., 23.584762573242188, 22.93895721435547, --],
         [--, --, --, ..., 1.5890443325042725, -8.162908554077148, --],
         [--, --, -0.8675096035003662, ..., 33.84625244140625,
          37.19544982910156, --],
         ...,
         [-9.631645202636719, -10.157059669494629, -10.559676170349121,
          ..., -17.144184112548828, -20.527921676635742, --],
         [-9.402924537658691, -9.904973030090332, -9.94972038269043,
          ..., -17.126976013183594, -21.53339195251465, --],
         [-10.571792602539062, -11.169559478759766, -11.204834938049316,
          ..., -17.020889282226562, -21.441513061523438, --]]],
  mask=[[[ True,  True,  True, ..., False, False,  True],
         [ True,  True,  True, ..., False, False,  True],
         [ True,  True, False, ..., False, False,  True],
         ...,
         [False, False, False, ..., False, False,  True],
         [False, False, False, ..., False, False,  True],
         [False, False, Fal